In [ ]:
# ---- CONFIGURATION ----
PATHOLOGY_REGIONS_PATH     = "path/to/pathology_regions_postQC.csv"
FINE_TYPED_DIR             = "path/to/fine_typed_objects/"  # directory containing annotated_<sampleID>.rds files
CELL_TYPE_ANNOTATIONS_PATH = "path/to/renamed_merfish_cell_types.csv"
# -----------------------


In [ ]:
require(tidyverse)
require(ggthemes)
require(harmony)
require(Seurat)
require(future)
require(furrr)
plan(multisession, workers = 10)
set.seed(1)

# load annotated pathology regions

In [ ]:
pathology_regions = data.table::fread(PATHOLOGY_REGIONS_PATH)
slice_sample(pathology_regions, n = 20)


# merge merfish objects

In [ ]:
merfish_files = list.files(path = FINE_TYPED_DIR, pattern = "annotated_.*.rds", full.names = TRUE)
merfish_files = merfish_files[grepl(pattern = paste(unique(pathology_regions$PatientID), collapse = '|'), x = merfish_files)]
length(merfish_files)
merfish_files %>% writeLines

In [ ]:
getCounts = function(f){
    temp = readr::read_rds(f)
    temp_counts = GetAssayData(temp, 'counts')
    return(temp_counts)
}

counts = future_map(merfish_files, getCounts)
merged_counts = do.call(cbind, counts)


# create seurat object

In [ ]:
merged_merfish = CreateSeuratObject(counts = merged_counts)
merged_merfish@meta.data$technology = 'MERFISH'
merged_merfish@meta.data$orig.ident = gsub(rownames(merged_merfish@meta.data), pattern = "_.*", replacement = "") %>% as.factor

# collect metadata

In [ ]:
merfish_annotations = data.table::fread(CELL_TYPE_ANNOTATIONS_PATH)



In [ ]:
temp = merfish_annotations %>%
    mutate(technology = 'MERFISH') %>%
    select(cell, orig.ident, ClusterTop, knn_renamed_cell_states, technology) %>%
    as.data.frame
rownames(temp) = temp$cell

temp = temp %>% filter(cell %in% rownames(merged_merfish@meta.data))


merged_merfish@meta.data$cell = rownames(merged_merfish@meta.data)

merged_merfish@meta.data = left_join(merged_merfish@meta.data, temp)
merged_merfish@meta.data$ClusterTop[is.na(merged_merfish@meta.data$ClusterTop)] = 'Mast'
merged_merfish@meta.data$knn_renamed_cell_states[is.na(merged_merfish@meta.data$knn_renamed_cell_states)] = 'Mast'

In [ ]:
merged_merfish@meta.data$orig.ident[merged_merfish@meta.data$orig.ident == 'G4659-CP-MET'] = 'G4659'

# normalize, scale, pca, 

In [ ]:
normFactor = merged_merfish@meta.data %>%
    select(orig.ident, nCount_RNA) %>%
    group_by(orig.ident) %>%
    summarize(medianCounts = median(nCount_RNA)) %>%
    pull(medianCounts) %>%
    median()


In [ ]:
options(future.globals.maxSize = 1e10)
plan(sequential)
merged_merfish = NormalizeData(merged_merfish, 
                          normalization.method = "LogNormalize", 
                          scale.factor = normFactor) %>%
    FindVariableFeatures() %>% 
    ScaleData() %>%
    RunPCA(npcs=50)


# cache

In [ ]:
readr::write_rds(merged_merfish, 'harmonized_merfish_20241105.rds')

# harmonize

In [ ]:
require(tidyverse)
require(ggthemes)
require(harmony)
require(Seurat)
set.seed(1)
merfish_hpca = readr::read_rds('merfish_hpca_20250212.rds')


In [ ]:
merged_merfish[['harmony']] = Seurat::CreateDimReducObject(
    embeddings = merfish_hpca,
    assay = 'RNA', 
    key = 'HPCA_', 
    global = TRUE
)

In [ ]:
require(scattermore)
ggplot() +
    geom_scattermost(as.matrix(Embeddings(merged_merfish, 'harmony')[, c('HPCA_1', 'HPCA_2')])) +
    ggpubr::theme_pubr()


In [ ]:
ggplot() +
    geom_scattermost(as.matrix(Embeddings(merged_merfish, 'harmony')[, c('HPCA_3', 'HPCA_4')])) +
    ggpubr::theme_pubr()

## cache

In [ ]:
readr::write_rds(merged_merfish, 'harmonized_merfish_20241105.rds')

# umap

In [ ]:
system.time({U = uwot::umap(Embeddings(merged_merfish, 'harmony'), 
               min_dist = 0.05, 
               spread = 0.30, 
               ret_extra = 'fgraph', 
               fast_sgd = TRUE)})
colnames(U$embedding) = c('HUMAP1', 'HUMAP2')

In [ ]:
rownames(U$fgraph) = colnames(U$fgraph) = rownames(merged_merfish@meta.data)
merged_merfish[['humap']] = Seurat::CreateDimReducObject(
    embeddings = U$embedding,
    assay = 'RNA', 
    key = 'HUMAP_', 
    global = TRUE
)

In [ ]:
new_graph = Seurat::as.Graph(U$fgraph)
DefaultAssay(new_graph) = DefaultAssay(merged_merfish)
merged_merfish[['humap_fgraph']] = new_graph

# cache

In [ ]:
readr::write_rds(merged_merfish, 'merged_harmonized_merfish_20241018.rds')

# plot by technology

In [ ]:
merged_merfish = readr::read_rds('merged_harmonized_merfish_20241018.rds')


In [ ]:
DimPlot(merged_merfish, 
        reduction = 'humap', 
        raster = TRUE, 
        group.by = 'technology', 
        shuffle = TRUE, 
        split.by = 'technology') +
theme_bw()

# plot by cell lineage

In [ ]:
DimPlot(object = merged_merfish %>% subset(technology == 'MERFISH'), 
        reduction = 'humap', 
        raster = TRUE, 
        shuffle = TRUE, 
        label = FALSE) +
        theme_bw()

In [ ]:
DimPlot(object = merged_merfish %>% subset(technology == 'MERFISH'),
        reduction = 'humap', 
        raster = TRUE, 
        group.by = 'ClusterTop', 
        shuffle = TRUE, 
        label = TRUE) +
        theme_bw()

In [ ]:
merged_merfish@meta.data$knn_renamed_cell_states[merged_merfish@meta.data$technology == 'MERFISH'] %>% table

In [ ]:
DimPlot(object = merged_merfish %>% subset(technology == 'MERFISH'),
        reduction = 'humap', 
        raster = TRUE, 
        group.by = 'knn_renamed_cell_states') +
theme_bw() +
theme(legend.position = 'none')